# Few-Shots Prompting

Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

## References:
* [Touvron et al. 2023](https://arxiv.org/pdf/2302.13971.pdf): present few shot properties  when models were scaled to a sufficient size
* [Kaplan et al., 2020](https://arxiv.org/abs/2001.08361)
* [Brown et al. 2020](https://arxiv.org/abs/2005.14165)


## Running this code on MyBind.org

Note: remember that you will need to **adjust CONFIG** with **proper URL and API_KEY**!

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/GenILab-FAU/prompt-eng/HEAD?urlpath=%2Fdoc%2Ftree%2Fprompt-eng%2Ffew_shots.ipynb)



In [1]:
##
## FEW SHOTS PROMPTING
##

from _pipeline import create_payload, model_req

phishing_message = "\"Congratulations! You’ve won a $500 Amazon gift card. Claim it here [Link].\""

#### (1) Adjust the inbounding  Prompt, simulating inbounding requests from users or other systems

MESSAGE = "Target Message for Analysis:" + phishing_message +"""
Instructions:
Classification: Determine whether the target message is "Phishing" or "Not Phishing."
Explanation: Provide a concise one-sentence explanation that highlights key indicators (e.g., unsolicited reward, urgent language, suspicious link) leading to your decision."
"""

#### (2) Adjust the Prompt Engineering Technique to be applied, simulating Workflow Templates
FEW_SHOT = """
You are a cybersecurity expert tasked with identifying phishing messages. Below are a few demonstration examples that illustrate the expected label distribution and input text format. Use these examples as guidance to analyze the target message.

Example 1:
Text Message: "Alert: Your account has been compromised. Please click [Link] immediately to secure your account."
Classification: Phishing
Explanation: The urgent tone and suspicious link strongly indicate a phishing attempt.

Example 2:
Text Message: "Your package has been delivered. Please check your mailbox for details."
Classification: Not Phishing
Explanation: The message provides clear, expected delivery information without any alarming language or links.

"""
PROMPT = FEW_SHOT + '\n' + MESSAGE 

#### (3) Configure the Model request, simulating Workflow Orchestration
# Documentation: https://github.com/ollama/ollama/blob/main/docs/api.md
payload = create_payload(target="ollama",
                         model="llama3.2:latest", 
                         prompt=PROMPT, 
                         temperature=1.0, 
                         num_ctx=100, 
                         num_predict=100)

### YOU DONT NEED TO CONFIGURE ANYTHING ELSE FROM THIS POINT
# Send out to the model
time, response = model_req(payload=payload)
print(response)
if time: print(f'Time taken: {time}s')

{'model': 'llama3.2:latest', 'prompt': '\nYou are a cybersecurity expert tasked with identifying phishing messages. Below are a few demonstration examples that illustrate the expected label distribution and input text format. Use these examples as guidance to analyze the target message.\n\nExample 1:\nText Message: "Alert: Your account has been compromised. Please click [Link] immediately to secure your account."\nClassification: Phishing\nExplanation: The urgent tone and suspicious link strongly indicate a phishing attempt.\n\nExample 2:\nText Message: "Your package has been delivered. Please check your mailbox for details."\nClassification: Not Phishing\nExplanation: The message provides clear, expected delivery information without any alarming language or links.\n\n\nTarget Message for Analysis:"Congratulations! You’ve won a $500 Amazon gift card. Claim it here [Link]."\nInstructions:\nClassification: Determine whether the target message is "Phishing" or "Not Phishing."\nExplanation

## How to improve it?

Following the findings from [Min et al. (2022)](https://arxiv.org/abs/2202.12837), here are a few more tips about demonstrations/exemplars when doing few-shot:

* "the label space and the distribution of the input text specified by the demonstrations are both important (regardless of whether the labels are correct for individual inputs)"
* the format you use also plays a key role in performance, even if you just use random labels, this is much better than no labels at all.
* additional results show that selecting random labels from a true distribution of labels (instead of a uniform distribution) also helps.